In [8]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create a synthetic dataset
X, y = make_classification(n_samples=200, n_features=4, n_informative=2, n_redundant=0, n_classes=2, random_state=42)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
import pennylane as qml

# Setup the quantum device and number of qubits
n_qubits = X_train.shape[1]
dev = qml.device("default.qubit", wires=n_qubits, shots=None)

# Define the quantum feature map using angle encoding
def feature_map(x):
    for i, val in enumerate(x):
        qml.RX(val, wires=i)

# Define the quantum kernel circuit
@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.expval(qml.Projector([0] * n_qubits, wires=range(n_qubits)))

In [10]:
def compute_kernel_row(i, X1, X2):
    return [kernel_circuit(X1[i], X2[j]) for j in range(X2.shape[0])]

def quantum_kernel_serial(X1, X2):
    m1 = X1.shape[0]
    K = np.zeros((m1, X2.shape[0]))
    for i in range(m1):
        K[i, :] = compute_kernel_row(i, X1, X2)
    return K

In [11]:
from joblib import Parallel, delayed

def quantum_kernel_parallel(X1, X2, n_jobs=-1):
    m1 = X1.shape[0]
    rows = Parallel(n_jobs=n_jobs, verbose=10)(
        delayed(compute_kernel_row)(i, X1, X2) for i in range(m1)
    )
    return np.array(rows)

In [12]:
import time

# Compute kernel matrix serially
start_time = time.time()
K_train_serial = quantum_kernel_serial(X_train, X_train)
K_test_serial = quantum_kernel_serial(X_test, X_train)
no_parallel_time = time.time() - start_time

# Compute kernel matrix in parallel
start_time = time.time()
K_train_parallel = quantum_kernel_parallel(X_train, X_train, n_jobs=-1)
K_test_parallel = quantum_kernel_parallel(X_test, X_train, n_jobs=-1)
parallel_time = time.time() - start_time

# Print results
print("=== Comparison of Precomputation Times ===")
print(f"No parallelization: {no_parallel_time:.4f} seconds")
print(f"Parallelization   : {parallel_time:.4f} seconds\n")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done 130 out of 160 | elapsed:    7.7s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done 147 out of 160 | elapsed:    7.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    8.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done   3 out of  40 | elapsed:  

=== Comparison of Precomputation Times ===
No parallelization: 38.3481 seconds
Parallelization   : 8.8711 seconds



[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.7s finished


In [13]:
from sklearn.svm import SVC

# Validate kernel matrices
assert np.allclose(K_train_serial, K_train_parallel, atol=1e-6)
assert np.allclose(K_test_serial, K_test_parallel, atol=1e-6)

# Train SVM using the parallel-computed kernel
qsvc = SVC(kernel='precomputed')
qsvc.fit(K_train_parallel, y_train)
accuracy = qsvc.score(K_test_parallel, y_test)

# Print accuracy
print(f"SVM Accuracy: {accuracy:.4f}")

SVM Accuracy: 0.8250
